In [ ]:
!git clone https://github.com/AmirJlr/K-Paths.git

In [ ]:
!pip install -r /content/K-Paths/requirements.txt

### Step 1: Download data GET the dataset:

In [ ]:
!gdown https://drive.google.com/uc?id=1_6meo_nB2RqHrVM9pqCBA67FQ6PR4QiI

In [ ]:
import zipfile

with zipfile.ZipFile('/content/data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/K-Paths')

In [ ]:
%cd /content/K-Paths

In [ ]:
!pwd

### Step 2: Create Augmented KG:

In [ ]:
!python k-paths/create_augmented_network.py

### Step 3: Extract K reasoning paths:

In [ ]:
!python k-paths/get-Kpaths.py \
  --dataset pharmaDB \
  --split train \
  --mode K-paths \
  --add_reverse_edges

In [ ]:
!python k-paths/get-Kpaths.py \
  --dataset pharmaDB \
  --split test \
  --mode K-paths \
  --add_reverse_edges


### Step 4: Create subgraphs for GNN input:

In [ ]:
!python k-paths/get-subgraph.py

# Zero-Shot LLM Inference and Evaluation

#### Inference (Use llm/llm_inference_v2.py for Tx-Gemma models)

In [ ]:
!python3 llm/llm_inference_v2.py \
--dataset_path data/paths/drugbank_test_add_reverse.json \
--dataset_name drugbank \
--output_dir outputs/drugbank \
--model_name_or_path google/txgemma-9b-chat \
--use_kg \
--debug

#### Evaluation

In [ ]:
!python llm/evaluate_llm_regex.py \
  --prediction_path output/google-txgemma-27b-chat-outputs-paths-drugbank_test_add_reverse-json-predictions.csv \
  --dataset drugbank \
  --model_style tx_gemma

# GNN Training & Evaluation

#### Train RGCN

In [ ]:
!python gnn/train.py \
  --seed "${SEED}" \
  --train_file_path path_to_your_train_set.csv \
  --hetionet_triplet_file path_to_hetionet.txt \
  --node_file path_to_node2id.json \
  --entity_drug_file path_to_BKG_entity2Id.json \
  --use_text_embeddings \
  --model_save_path "trained_model_seed${SEED}.pt"


#### Evaluate Trained Model

(Note: Most optional arguments (e.g., --embedding_dim, --log_file, --output_predictions) have sensible defaults.)

In [ ]:
!python gnn/eval.py \
  --model_path "trained_model_seed${SEED}.pt" \
  --train_file_path path_to_your_train_set.csv \
  --test_file path_to_your_test_set.csv \
  --hetionet_triplet_file path_to_hetionet.txt \
  --node_file path_to_node2id.json \
  --use_text_embeddings \